## 트리의 앙상블

### 랜덤포레스트
결정 트리를 랜덤하게 만들어 결정 트리의 **숲**을 만듦 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듦<br>
데이터 세트에서 중복을 허용하여 데이터를 샘플링하는데 이것을 **부트스트랩**이라 함<br>
![랜덤포레스트](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F5IHJQ%2FbtqXRvIzVQi%2FFAQNXxLkfn9xA69nun9qZ1%2Fimg.png)<br>
분류: 각 트리의 클래스별 확률을 평가하여 가장 높은 확률을 가진 클래스를 예측으로 삼음<br>
회귀: 단순히 각 트리의 예측을 평균<br><br>
훈련 세트에 과대적합되는 것을 막고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('wine.csv')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #과대적합

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
#앞에서 했던 '결정트리'에서 만든 특성 중요도와 비교할 때 당도의 중요도는 떨어지고 다른 특성의 중요도는 올라감
#하나의 특성에 과도하게 집중하지 않고 좀 더 많으 특성이 훈련에 기여할 기회를 얻음

[0.23167441 0.50039841 0.26792718]


자체적으로 모델을 평가하는 점수를 얻을 수 있음<br>
부트스트랩 샘플에 포함되지 않고 남는 샘플을 **OOB**샘플이라고 함<br>
OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음

In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_) #교차검증에서 얻은 점수와 매우 비슷한 결과를 얻음

0.8934000384837406


### 엑스트라트리
부트스트랩을 사용하지 않는다는 점에서 차이가 나옴 즉, 전체 훈련 세트를 사용함<br>
그러나 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아닌 무작위로 분할! => 과대적합 막고 검증 세트의 점수를 높이는 효과가 있음<br>

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

#랜덤하게 노드를 분할하여 계산 속도가 빠름
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_) #결정 트리보다 당도에 대한 의존성이 작음

[0.20183568 0.52242907 0.27573525]


### 그레이디언트 부스팅
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법<br>
기본적으로 깊이가 3인 결정 트리를 100개 사용, 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #과대적합이 되지 않음!

0.8881086892152563 0.8720430147331015


In [11]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)#결정 트리 개수를 500개로 늘림
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #과대적합 잘 억제

0.9464595437171814 0.8780082549788999


In [13]:
#순서대로 트리를 추가하여 훈련 속도는 느린편(n_jobs 매개변수가 없음)
gb.fit(train_input, train_target)
print(gb.feature_importances_) #랜덤 포레스트보다 일부 특성(당도)에 더 집중함

[0.15872278 0.68011572 0.16116151]


### 히스토그램 기반 부스팅
입력 특성을 256개의 구간으로 나눔(최적의 분할을 매우 빠르게 찾을 수 있음)<br>
256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용 그래서 누락된 특성이 있더라도 따로 전처리할 필요 없음!<br>

In [14]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))#그레이디언트보다 조금 더 높은 성능 제공

0.9321723946453317 0.8801241948619236


In [15]:
#permutation_importance() 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 더 중요한 지 계산
from sklearn.inspection import permutation_importance

# 훈련 세트에서 특성 중요도 계산
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean) #특성 중요도 평균 출력, 랜덤 포레스트와 비슷

[0.08876275 0.23438522 0.08027708]


In [16]:
# 테스트 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)#조금 더 당도에 집중

[0.05969231 0.20238462 0.049     ]


In [17]:
hgb.score(test_input, test_target) #약 87% 정확도 얻음

0.8723076923076923

사이킷런 말고도 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여럿 있음<br>
대표적으로 **XGBoost**와 **LightGBM**이 있음

In [19]:
pip install xgboost

     ------------------------------------- 125.4/125.4 MB 34.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [22]:
pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 10.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [23]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
